# nasb ketabkhane ha

1.   List item
2.   List item



In [17]:
pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=4fc759d2248931010957c18cb16d702950e23d5a3ec2f2029a4b6c66be4df7ac
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
!pip install transformers torch torchvision torchaudio

# estekhraje khabar

In [ ]:
import requests
from bs4 import BeautifulSoup
import random

url = 'https://mashreghnews.ir/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

links = []

#  estekhraje link ba olgooye:   /news/{y}/{x}
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if '/news/' in href:
        links.append(href)

# entekhabe 5 link be tore random
selected_links = random.sample(links, 5)

# zakhire kardane mohtaviate link ha
for link in selected_links:
    full_url = f'https://mashreghnews.ir{link}'
    response = requests.get(full_url)
    content_soup = BeautifulSoup(response.text, 'html.parser')

    # estekhraje matne khabar: itemprop="articleBody"
    article_body = content_soup.find('div', {'itemprop': 'articleBody'})

    # entekhabe yek nam baraye khabar
    file_name = f'content_{random.randint(1, 1000)}.txt'

    # zakhire sazi khabar
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(article_body.get_text())


# ejraye model

In [32]:
import os
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import re

# Load kardane model
model_name = "HooshvareLab/bert-fa-base-uncased-clf-persiannews"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

def preprocess_text(text):
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Replace newlines with a space
    text = text.replace('\n', ' ')

    return text


# sakhte pipeline baraye pardazesh voroodi
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# moarefiye nam va mahale zakhire sazi file ha
file_names = [file for file in os.listdir() if file.endswith(".txt")]

# khandane file ha va classify kardane an ha
for file_name in file_names:
    with open(file_name, 'r', encoding='utf-8') as file:
        original_text = file.read()
        text = preprocess_text(original_text)

        # bardashtane tanha 512 token avale matn be dalil mahdoodiat bert
        text_for_classification = tokenizer.encode(text, max_length=510, truncation=True, return_tensors='pt')
        # tabdile token ha be kalame
        text_for_classification_str = tokenizer.decode(text_for_classification[0])

        # Classify kardane matn
        result = classifier(text_for_classification_str)

        # namayesh khorooji
        print(f"File: {file_name}")
        print('text: ',text[:100] ,' ...')

        print(f"Label: {result[0]['label']} | Score: {result[0]['score']:.4f}\n")


File: content_541.txt
text:   به گزارش مشرق، در پی حمله تروریستی به مقر فرماندهی انتظامی شهرستان راسک، امیر سرلشکر سیدعبدالرحیم م  ...
Label: سیاسی | Score: 0.9918

File: content_769.txt
text:  سرویس فرهنگ و هنر مشرق - این روزها خبر تعطیلی قریب‌الوقوع شبکه «من و تو» غیر از بازتاب‌های سیاسی در   ...
Label: علمی فناوری | Score: 0.9146

File: content_819.txt
text:  به گزارش مشرق، مهدی بختیاری طی یادداشتی در روزنامه وطن امروز نوشت: در جریان حمله تروریستی -‌بامداد د  ...
Label: سیاسی | Score: 0.8752

File: content_587.txt
text:   به گزارش مشرق، محمد باقر قالیباف رئیس مجلس شورای اسلامی پس از رد کلیات بودجه سال ۱۴۰۳ گفت: براساس آ  ...
Label: سیاسی | Score: 0.9524

File: content_524.txt
text:   به گزارش مشرق، تیری آنری ستاره سابق بارسلونا و آرسنال به کیلیان امباپه هموطنش نصحیت کرد که به رئال   ...
Label: ورزشی | Score: 0.9995

File: content_169.txt
text:  به گزارش مشرق، علی جعفری، فعال رسانه، با انتشار عکسی در اینستاگرام خود نوشت: روز اول آتش‌بس از خانم   ...
Label: اجتماعی | Score: 0.9645

Fi